<a href="https://colab.research.google.com/github/moharamfatema/graduation-project/blob/main/src/data-preprocessing/Frames_Extractor(28_28)_aug_with_overlap_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install npy_append_array

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import cv2
import numpy as np
from npy_append_array import NpyAppendArray
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_SHAPE = (28,28,28)
ROOT = '/content/drive/MyDrive/Datasets/LAD-2000'
out = '/content/drive/MyDrive/Datasets/LAD-2000/Extracted_Frames'

In [ ]:
def extract_frames(root,
                   label,
                   out,
                   packet_shape=INPUT_SHAPE,
                   n_class_packets_threshold=None,
                   overlap=0):
  '''
    summary:
    transform videos located at root into 2 numpy arrays [X, y] loacated at out
  '''
  # initiate packets counter
  n_class_packets = 0

  # initiate packet
  packet = np.empty(packet_shape,
                    dtype=np.uint8)

  # unique packet frames count
  n_unique_frames = packet_shape[0] - overlap

  # intiate extracted frames X ndarray
  with NpyAppendArray(out + 'X.npy') as npaa:

    # loop on videos locations
    for subdir, _, videos_paths in [*os.walk(root)]:
      for video_path in videos_paths:
        #print(video_path)
        # initate capture
        cap = cv2.VideoCapture(subdir + os.sep + video_path)

        # print('full_path', subdir + os.sep +  video_path)

        # TODO: Reformulate considering overlaps
        # Calculate packets per video
        n_video_packets = (int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - overlap) // n_unique_frames
        # print(n_video_packets)
        if n_video_packets <= 0:
          continue

        # accumulate class whole packets count
        n_class_packets += n_video_packets


        # initiate video packets array
        video_packets = np.empty((n_video_packets,) + packet_shape,
                                dtype=np.uint8)


        for j in range(packet_shape[0]):

          # read the frame
          ret, frame = cap.read()

          # NOTE: is it the best interpolation?
          # resize frame shape
          # if ret:

          frame = cv2.resize(frame, packet_shape[1:], interpolation=cv2.INTER_AREA)

          # assure grayscale
          if len(frame.shape) > 2:
              frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

          # assign to packet
          packet[j] = frame

        # assign to video packets
        video_packets[0] = packet

        # loop on video frames
        for i in range(1, n_video_packets):

          if overlap > 0:
            packet[:overlap] = packet[-overlap:]
          # packet[:overlap] = packet[-overlap:] if overlap > 0 else packet[:overlap]
          # loop on packet size
          for j in range(overlap, packet_shape[0]):
              # print(j)
              # read the frame
              ret, frame = cap.read()
              #print(frame.dtype)
              # NOTE: is it the best interpolation?
              # resize frame shape
              # if ret:
              frame = cv2.resize(frame, packet_shape[1:], interpolation=cv2.INTER_AREA)

              # assure grayscale
              if len(frame.shape) > 2:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

              # assign to packet
              packet[j] = frame

          # assign to video packets
          video_packets[i] = packet

        cv2.destroyAllWindows()

        # print(video_packets.shape)
        # save video packets
        npaa.append(video_packets)

        if n_class_packets_threshold and n_class_packets >= n_class_packets_threshold:
          break
      if n_class_packets_threshold and n_class_packets >= n_class_packets_threshold:
        break

  print('for', label, n_class_packets)
  # save labels array
  y = np.full((n_class_packets, 1), label, dtype=np.uint8)
  with NpyAppendArray(out + 'y.npy') as npaa:
    npaa.append(y)

# New Section

In [ ]:
len([f for f in os.listdir(ROOT + '/Violent/train/Crash') if os.path.isfile(os.path.join(ROOT + '/Violent/train/Crash', f))])

44

In [ ]:
def write_labels(out, label, n_class_packets):
    y = np.full((n_class_packets, 1), label, dtype=np.uint8)
    with NpyAppendArray(out + 'y.npy') as npaa:
        npaa.append(y)

In [ ]:
# train
write_labels('/home/azureuser/Datasets/Extracted-Frames/Train/', 0, 156032 - 35744)
write_labels('/home/azureuser/Datasets/Extracted-Frames/Train/', 1, 35744)

In [ ]:
# test
write_labels('/home/azureuser/Datasets/Extracted-Frames/Test/', 0, 39187 - 7939)
write_labels('/home/azureuser/Datasets/Extracted-Frames/Test/', 1, 7939)

In [ ]:
# val
write_labels('/home/azureuser/Datasets/Extracted-Frames/Val/', 0, 46486 - 8588)
write_labels('/home/azureuser/Datasets/Extracted-Frames/Val/', 1, 8588)

# New Section

In [ ]:
#extract_frames(ROOT + 'Non-Violent/train/', 0, out + 'Train/')

In [ ]:
# train
extract_frames(ROOT + 'Non-Violent/train/', 0, out + 'Train/')


for 0 64053


In [ ]:
extract_frames(ROOT + 'Violent/train/', 1, out + 'Train/', overlap=27)

for 1 517609


In [ ]:
X = np.load('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Train/X.npy')
y = np.load('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Train/y.npy')

In [ ]:
X.shape

(46535, 28, 28, 28)

In [ ]:
# test
extract_frames(ROOT + 'Non-Violent/test/', 0, out + 'Test/')
extract_frames(ROOT + 'Violent/test/', 1, out + 'Test/', overlap=21)

KeyboardInterrupt: ignored

In [ ]:
X_test = np.load('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Test/X.npy')
y_test = np.load('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Test/y.npy')



In [ ]:
X_test.shape

(12200, 28, 28, 28)

In [ ]:
del X_test, y_test

In [ ]:
X = np.concatenate((X,X_test))
y= np.concatenate((y,y_test))
print(X.shape)
print(y.shape)

(58735, 28, 28, 28)
(58735, 1)


In [ ]:
# val
#extract_frames(ROOT + 'Non-Violent/val/', 0, out + 'Val/')
#extract_frames(ROOT + 'Violent/val/', 1, out + 'Val/')

for 0 9643
for 1 4175


In [ ]:
X_val = np.load('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Val/X.npy')
y_val = np.load('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Val/y.npy')


In [ ]:
X = np.concatenate((X, X_val))
y = np.concatenate((y,y_val))
print(X.shape)
print(y.shape)

(72553, 28, 28, 28)
(72553, 1)


In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
np.save('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Shuffled/Train/X', X_train)
np.save('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Shuffled/Train/y', y_train)

In [ ]:
np.save('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Shuffled/Test/X', X_test)
np.save('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Shuffled/Test/y', y_test)

In [ ]:
np.save('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Shuffled/Val/X', X_val)
np.save('/content/drive/MyDrive/All extracted frames/UCF-crime 28/Shuffled/Val/y', y_val)

In [ ]:
for x in range(1, 0):
  print(x)

In [ ]:
x = [1, 2, 3]
x[:0]

[]